In [1]:
import librosa
import numpy as np

AUDIO_PATH = "../audio/downloaded/test.wav"

audio, sr = librosa.load(AUDIO_PATH, sr=16000)

print("Audio loaded")
print("Sample rate:", sr)
print("Audio length (seconds):", len(audio) / sr)

c:\Users\nikhi\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\nikhi\AppData\Local\Temp\ipykernel_15296\231573709.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(AUDIO_PATH, sr=16000)
c:\Users\nikhi\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: '../audio/downloaded/test.wav'

In [2]:
from faster_whisper import WhisperModel

MODEL_PATH = r"C:\whisper_models\faster-whisper-small"

model = WhisperModel(
    MODEL_PATH,
    device="cpu",
    compute_type="int8"
)

segments, info = model.transcribe(
    AUDIO_PATH,
    word_timestamps=True
)

segments = list(segments)

FileNotFoundError: [Errno 2] No such file or directory: '../audio/downloaded/test.wav'

In [ ]:
words = []

for segment in segments:
    if segment.words is None:
        continue
    for w in segment.words:
        words.append({
            "word": w.word,
            "start": w.start,
            "end": w.end
        })

print("Total words:", len(words))
print(words[:5])

In [3]:
def get_audio_segment(audio, sr, start, end):
    start_sample = int(start * sr)
    end_sample = int(end * sr)
    return audio[start_sample:end_sample]

In [4]:
test_word = words[0]
segment_audio = get_audio_segment(
    audio,
    sr,
    test_word["start"],
    test_word["end"]
)

print("Segment length (samples):", len(segment_audio))
print("Segment length (seconds):", len(segment_audio) / sr)

NameError: name 'words' is not defined

In [5]:
def extract_features(segment_audio, sr):
    features = {}

    # Duration
    features["duration"] = len(segment_audio) / sr

    # RMS Energy
    rms = librosa.feature.rms(y=segment_audio)
    features["rms_mean"] = np.mean(rms)

    # Zero Crossing Rate
    zcr = librosa.feature.zero_crossing_rate(segment_audio)
    features["zcr_mean"] = np.mean(zcr)

    # MFCCs
    mfcc = librosa.feature.mfcc(y=segment_audio, sr=sr, n_mfcc=13)
    for i in range(13):
        features[f"mfcc_{i}_mean"] = np.mean(mfcc[i])

    return features

In [6]:
features = extract_features(segment_audio, sr)
print(features)

NameError: name 'segment_audio' is not defined

In [7]:
word_features = []

for w in words:
    segment_audio = get_audio_segment(
        audio, sr, w["start"], w["end"]
    )

    if len(segment_audio) < 200:  # skip very tiny segments
        continue

    feats = extract_features(segment_audio, sr)

    word_features.append({
        "word": w["word"],
        "start": w["start"],
        "end": w["end"],
        **feats
    })

print("Extracted features for", len(word_features), "words")

NameError: name 'words' is not defined